# Imports and Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU, ReLU

from keras.applications import EfficientNetB5

USE_GPU = True

2024-03-07 20:59:31.537306: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 20:59:31.565347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 20:59:31.565378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 20:59:31.566185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 20:59:31.571710: I tensorflow/core/platform/cpu_feature_guar

In [2]:
if USE_GPU:
    #tf.debugging.set_log_device_placement(True)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



2024-03-07 20:59:32.778744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:32.800152: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:32.800233: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:32.963705: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:32.963780: I external/local_xla/xla/stream_executor

In [3]:
## Define path to data and parameters for loader
TRAIN_PATH = './data/training/'
TEST_PATH = './data/testing/'
LABELS_PATH = './data/training_labels.csv'

TEST_SIZE = 0.1
BATCH_SIZE = 16
TARGET_SIZE = (256, 256)

N_CLASSES = 5

## Encoding labels
LABEL_ENCODING = {
    'daisy': 0,
    'dandelion': 1,
    'rose': 2,
    'sunflower': 3,
    'tulip': 4
}

Read in data and assign labels

In [4]:
## Get labels for train and test data
labels_df = pd.read_csv(LABELS_PATH)
labels_df['full_id'] = [os.path.join('/home/scottbrown/byu/stat486/projects/flower-image-classification/data/training/training', l) for l in labels_df.ID]

In [5]:
## NOTE: `flow_from_dataframe` via ImageDataGenerator is deprecated
# In the future, organize directory structure for `flow_from_directory`
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=TEST_SIZE)
train_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    target_size=TARGET_SIZE,
    subset='training',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

test_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    target_size=TARGET_SIZE,
    subset='validation',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

## Read in test data to predict on
new_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
new_gen = new_datagen.flow_from_directory(
    TEST_PATH, 
    target_size=TARGET_SIZE, 
    class_mode='categorical', 
    shuffle=False, 
    batch_size=1)

class_names = list(train_gen.class_indices.keys())

Found 3109 validated image filenames belonging to 5 classes.
Found 345 validated image filenames belonging to 5 classes.
Found 863 images belonging to 1 classes.


# Preview images

In [6]:
# ## Recreate the above code for train_gen generator object
# class_names = list(train_gen.class_indices.keys())
# for images, labels in train_gen:
#   plt.figure(figsize=(10, 10))
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     lab = class_names[labels[i].argmax()]
#     plt.imshow(images[i])
#     plt.title(lab)
#     plt.axis("off")
#   break

# Create model

In [7]:
# base_model = EfficientNetB5(weights='imagenet', include_top=False, drop_connect_rate=0.4, input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))
# UNFREEZE_N = 2

# # Freeze all but N layers in the base model
# for layer in base_model.layers:
#     layer.trainable = False

# # Optionally, unfreeze the top N layers
# for layer in base_model.layers[-UNFREEZE_N:]:
#     layer.trainable = True

# #x = base_model.output

In [ ]:
cnn = tf.keras.Sequential([
    ## Convolutional => MaxPooling layers
    tf.keras.layers.Conv2D(32, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(32, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),

    ## Fully connected layers
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    # ## Output layer
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])

In [8]:
cnn = tf.keras.Sequential([
    ## Use base model for transfer learning
    #base_model,

    ## 3 Convolutional => MaxPooling layers
    tf.keras.layers.Conv2D(32, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(32, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),

    # tf.keras.layers.Conv2D(32, (2, 2)),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.ReLU(),
    # #tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.4),

    # ## 1 Fully connected layer
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    # ## Output layer
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])

# cnn = tf.keras.Sequential([
#     ## Use base model for transfer learning
#     base_model,

#     ## 3 Convolutional => MaxPooling layers
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.4),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(32, activation='relu'),
    
#     tf.keras.layers.Dense(N_CLASSES, activation='softmax')
# ])

2024-03-07 20:59:33.087176: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:33.087356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:33.087401: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:33.087724: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:59:33.087766: I external/local_xla/xla/stream_executor

In [9]:
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
# for layer in base_model.layers:
#     layer.trainable = False

# x = base_model.output
# x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# predictions = Dense(4, activation='softmax')(x)

# model_transfer = Model(inputs=base_model.input, outputs=predictions)

# model_transfer.compile(optimizer='adam',
#                        loss='categorical_crossentropy',
#                        metrics=['accuracy'])

# model_transfer.fit(train_generator, validation_data=validation_generator, epochs=10)


In [10]:
cnn.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Fit model

In [11]:
checkpoint_filepath = '/tmp/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

cnn.fit( 
  train_gen,
  validation_data=test_gen,
  callbacks=[model_checkpoint_callback],
  epochs=50
)

2024-03-07 20:59:33.362735: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-07 20:59:33.440864: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-07 20:59:34.125492: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/50


2024-03-07 20:59:35.296453: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-07 20:59:36.030215: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fba793c7ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-07 20:59:36.030245: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-03-07 20:59:36.033955: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709870376.098167   25949 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


195/195 [==============================] - 18s 74ms/step - loss: 1.2609 - accuracy: 0.4793 - val_loss: 1.5286 - val_accuracy: 0.3710
Epoch 2/50
195/195 [==============================] - 13s 64ms/step - loss: 0.9212 - accuracy: 0.6423 - val_loss: 1.1844 - val_accuracy: 0.5246
Epoch 3/50
195/195 [==============================] - 12s 64ms/step - loss: 0.6718 - accuracy: 0.7581 - val_loss: 1.1105 - val_accuracy: 0.5478
Epoch 4/50
195/195 [==============================] - 12s 64ms/step - loss: 0.3846 - accuracy: 0.8717 - val_loss: 1.2774 - val_accuracy: 0.5536
Epoch 5/50
195/195 [==============================] - 12s 64ms/step - loss: 0.2178 - accuracy: 0.9360 - val_loss: 1.2145 - val_accuracy: 0.5681
Epoch 6/50
195/195 [==============================] - 12s 64ms/step - loss: 0.1323 - accuracy: 0.9653 - val_loss: 1.1598 - val_accuracy: 0.6145
Epoch 7/50
195/195 [==============================] - 12s 63ms/step - loss: 0.0961 - accuracy: 0.9759 - val_loss: 1.6154 - val_accuracy: 0.5014
Epo

In [12]:
test_eval = cnn.evaluate(test_gen)#X_test, y_test, verboseq0)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

22/22 [==============================] - 0s 20ms/step - loss: 1.6783 - accuracy: 0.5855
Test loss: 1.6783088445663452
Test accuracy: 0.5855072736740112


# Predict new data

In [13]:
y_pred = cnn.predict(new_gen)

863/863 [==============================] - 3s 2ms/step


In [14]:
pred_files = [f.split('/')[-1] for f in new_gen.filenames]
pred_labels = [class_names[i] for i in y_pred.argmax(axis=1)]
submission_df = pd.DataFrame({'ID': pred_files, 'Prediction': pred_labels})

In [15]:
## Write to CSV
submission_df.to_csv('submission.csv', index=False)